In [2]:
import os
import torch
import numpy as np
import plotly
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit

from botorch import fit_gpytorch_model
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.models import SingleTaskGP, ModelListGP
from botorch.models.multitask import MultiTaskGP

from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood

from botorch.optim import optimize_acqf

from pycaret.regression import *

c:\Users\99823827\Anaconda3\envs\Dissertacao_nightly\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\99823827\Anaconda3\envs\Dissertacao_nightly\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\99823827\Anaconda3\envs\Dissertacao_nightly\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\99823827\Anaconda3\envs\Dissertacao_nightly\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [3]:
df=pd.read_csv("..\data_gen\data_to_modeling.csv",sep='\t',dtype='float',index_col=0)
df

,depth,vc,rake_angle,clearance_angle,expected_cut_force,A,B,C,n,m,d1,d2,d3,d4,d5,densidade,young_modulus,poisson_coef
0.0,0.5,3000.0,-10.0,5.0,6063.298535,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269
1.0,0.5,3000.0,-7.0,5.0,4420.143506,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269
2.0,0.5,3000.0,-4.0,5.0,5492.385645,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269
3.0,0.5,3000.0,-1.0,5.0,4820.064746,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269
4.0,0.5,3000.0,2.0,5.0,3987.947656,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319.0,0.5,4500.0,2.0,5.0,4818.066699,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269
320.0,0.5,4500.0,5.0,5.0,3898.750684,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269
321.0,0.5,4500.0,8.0,5.0,5058.776172,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269
322.0,0.5,4500.0,11.0,5.0,5383.911328,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269


In [7]:
df_material=pd.DataFrame([["AISI_1020",0.5,99999,99999,5,9999999,213,53,0.0055,0.34,0.81,2.70E-03,206000,0.28],
              ["AISI_1045",0.5,99999,99999,5,9999999,553,600,0.0134,0.234,1.000,2.70E-03,210000,0.269],
              ["AISI_4340",0.5,99999,99999,5,9999999,792,510,0.014,1.030,0.26,2.70E-03,209000,0.28],
              ["AISI_4340_52_HRC",0.5,99999,99999,5,9999999,1638,1034,0.0058,0.46,1.003,2.70E-03,222000,0.29],
              ["STEEL_304",0.5,99999,99999,5,9999999,310,1000,0.07,0.65,1.000,2.70E-03,209000,0.28],
              ["STEEL_52100",0.5,99999,99999,5,9999999,774.78,134,0.0018,0.37,3.171,2.70E-03,222000,0.29]],
              columns=["material","depth","vc","rake_angle","clearance_angle","expected_cut_force",
                        "A","B","C","n","m","densidade","young_modulus","poisson_coef"])
df_material

,material,depth,vc,rake_angle,clearance_angle,expected_cut_force,A,B,C,n,m,densidade,young_modulus,poisson_coef
0,AISI_1020,0.5,99999,99999,5,9999999,213.00,53,0.0055,0.340,0.810,0.0027,206000,0.280
1,AISI_1045,0.5,99999,99999,5,9999999,553.00,600,0.0134,0.234,1.000,0.0027,210000,0.269
2,AISI_4340,0.5,99999,99999,5,9999999,792.00,510,0.0140,1.030,0.260,0.0027,209000,0.280
3,AISI_4340_52_HRC,0.5,99999,99999,5,9999999,1638.00,1034,0.0058,0.460,1.003,0.0027,222000,0.290
4,STEEL_304,0.5,99999,99999,5,9999999,310.00,1000,0.0700,0.650,1.000,0.0027,209000,0.280
5,STEEL_52100,0.5,99999,99999,5,9999999,774.78,134,0.0018,0.370,3.171,0.0027,222000,0.290


In [12]:
df_v2=df.merge(df_material[["material","A","B","C","n","m"]],on=["A","B","C","n","m"],how="left")

In [14]:
df_v2.head()

,depth,vc,rake_angle,clearance_angle,expected_cut_force,A,B,C,n,m,d1,d2,d3,d4,d5,densidade,young_modulus,poisson_coef,material
0,0.5,3000.0,-10.0,5.0,6063.298535,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269,AISI_1045
1,0.5,3000.0,-7.0,5.0,4420.143506,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269,AISI_1045
2,0.5,3000.0,-4.0,5.0,5492.385645,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269,AISI_1045
3,0.5,3000.0,-1.0,5.0,4820.064746,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269,AISI_1045
4,0.5,3000.0,2.0,5.0,3987.947656,553.0,600.0,0.0134,0.234,1.0,0.06,3.31,-1.96,0.0018,0.58,2.700000e-09,210000.0,0.269,AISI_1045


## Otimização usando Bayesian Optimization com Botorch em cima dos dados crus

In [4]:
def get_next_points(init_x,init_y,best_init_y,bounds,n_points=1):
  single_model=SingleTaskGP(init_x,init_y)
  mll=ExactMarginalLogLikelihood(single_model.likelihood,single_model)

  #multi_input_model=MultiTaskGP(init_x,init_y,task_feature=0)
  #mll=ExactMarginalLogLikelihood(multi_input_model.likelihood,multi_input_model)

  #bounds = torch.tensor([[-10.],[10.]])
  #single_model=SingleTaskGP(init_x,init_y)
  #mll=ExactMarginalLogLikelihood(single_model.likelihood,single_model)

  fit_gpytorch_model(mll)
  
  EI = qExpectedImprovement(model=single_model, best_f=best_init_y)

  candidates, _ = optimize_acqf(
      acq_function=EI,
      bounds=bounds,
      q=n_points,
      num_restarts=200,
      raw_samples=1024,
      options={"batch_limit":5,"maxiter":200}
  )

  return candidates

In [27]:
df_v2["material"].unique()

array(['AISI_1045', 'AISI_1020', 'STEEL_304', 'AISI_4340', 'STEEL_52100',
       'AISI_4340_52_HRC'], dtype=object)

In [47]:
from optimizers.objective_functions.objective_functions import *

model_lgbm_0=load_model("models/lgbm_v0")
model_lgbm_1=load_model("models/lgbm_v1")
model_lasso_0=load_model("models/lasso_v0")
model_lasso_1=load_model("models/lasso_v1")

def build_input(controlable_input, fixed_input,construct_dict):
    final_input_df=pd.DataFrame()
    for input in controlable_input:
        for counter,parameter in enumerate(construct_dict.keys()):
            fixed_input[parameter]=input[counter]
        final_input_df=pd.concat([final_input_df,fixed_input])
    return final_input_df

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [53]:
bounds = torch.tensor([[3000.,-10],[5000.,10]])
results={"material":[],"qtd_dados":[],"best_y":[],"new_candidates_vc":[],"new_candidates_rake_angle":[],"results_new_candidates":[]}
construct_dict={"rake_angle":{"sequence_len":4,"max_bound":10,"lower_bound":-10,"type":"int","neg_allowed":True},
                "vc":{"sequence_len":10,"max_bound":4500,"lower_bound":3000,"type":"int","neg_allowed":False},
                #"depth":{"sequence_len":4,"max_bound":2,"lower_bound":0.1,"type":"float"}
                }

for material in df_v2["material"].unique():
    for model in ["models/lgbm_v0","models/lgbm_v1","models/lasso_v0","models/lasso_v1"]:

        df_opt=df_v2[df_v2["material"]==material]
        candidates_list=[]
        cv=TimeSeriesSplit(10)
        best_list=[]
        for train, test in cv.split(df_opt[["vc","rake_angle","A","B","C","n","m"]]):
            #ToDo: colocar mais um loop para iterara na otimização até atingir um X de iterações sem melhoria
            init_x=torch.tensor(df_opt.iloc[train][["vc","rake_angle"]].values, dtype=torch.double)

            init_y=torch.tensor(-df_opt.iloc[train]["expected_cut_force"].values,dtype=torch.double)
            init_y=init_y.unsqueeze(-1)
            best_init_y=init_y.max().item()

            new_candidates = get_next_points(init_x, init_y, best_init_y, bounds, 1)

            #
            obj_func=objective_function(path=model,model="pycaret",maximize=False)            
            inputs=np.array([[new_candidates.numpy()[0][1],new_candidates.numpy()[0][0]]])
            pd_inputs=build_input(inputs,df_material[df_material["material"]==material],construct_dict)
            ##
            fitness=obj_func.predict(X=pd_inputs)

            results["material"].append(material)
            results["qtd_dados"].append(len(train))
            results["best_y"].append(best_init_y)
            results["new_candidates_vc"].append(new_candidates.numpy()[0][0])
            results["new_candidates_rake_angle"].append(new_candidates.numpy()[0][1])
            results["results_new_candidates"].append(fitness)
            
            print(f"Qtd de dados: {len(train)} | melhor ponto até agora : {best_init_y} |New candidates are: {new_candidates}")

            best_list.append(best_init_y)
            candidates_list.append(new_candidates)

    print(material)
    print(candidates_list)
    print(best_list)

Transformation Pipeline and Model Successfully Loaded
Qtd de dados: 14 | melhor ponto até agora : -3822.69765625 |New candidates are: tensor([[4502.7344,    4.7032]])
Transformation Pipeline and Model Successfully Loaded
Qtd de dados: 27 | melhor ponto até agora : -3822.69765625 |New candidates are: tensor([[3848.1018,    9.8066]])
Transformation Pipeline and Model Successfully Loaded
Qtd de dados: 40 | melhor ponto até agora : -3822.69765625 |New candidates are: tensor([[3183.3220,    3.7498]])
Transformation Pipeline and Model Successfully Loaded
Qtd de dados: 53 | melhor ponto até agora : -3784.288183593752 |New candidates are: tensor([[3017.0281,    4.9513]])
Transformation Pipeline and Model Successfully Loaded
Qtd de dados: 66 | melhor ponto até agora : -3673.538818359376 |New candidates are: tensor([[4.9262e+03, 6.4598e-01]])
Transformation Pipeline and Model Successfully Loaded
Qtd de dados: 79 | melhor ponto até agora : -3673.538818359376 |New candidates are: tensor([[ 3.5177e

In [39]:
df_v2[(df_v2["material"]=="AISI_1020")
      &(df_v2["vc"]>=4500)
      &(df_v2["vc"]<=5000)
      &(df_v2["rake_angle"]>=2)
      &(df_v2["rake_angle"]<=5)]

,depth,vc,rake_angle,clearance_angle,expected_cut_force,A,B,C,n,m,d1,d2,d3,d4,d5,densidade,young_modulus,poisson_coef,material
103,0.5,4500.0,2.0,5.0,751.797424,213.0,53.0,0.0055,0.34,0.81,0.05,3.44,-2.12,0.002,0.61,2.700000e-09,206000.0,0.28,AISI_1020
104,0.5,4500.0,5.0,5.0,1054.731384,213.0,53.0,0.0055,0.34,0.81,0.05,3.44,-2.12,0.002,0.61,2.700000e-09,206000.0,0.28,AISI_1020


In [54]:
pd.DataFrame(results).sort_values("best_y",ascending=False).drop_duplicates("material",keep="first")

,material,qtd_dados,best_y,new_candidates_vc,new_candidates_rake_angle,results_new_candidates
57,AISI_1020,27,-725.455286,4113.220215,2.493080,[-823.8302930690562]
119,STEEL_304,33,-1807.543677,3405.462891,-3.611793,[-2224.746189960162]
169,STEEL_52100,33,-2656.604883,4390.728027,-4.202898,[-2880.661555157618]
9,AISI_1045,131,-3666.605664,4557.784668,-8.096388,[-5607.939498807516]
138,AISI_4340,30,-5066.548047,3732.188721,8.992447,[-6345.938661814821]
209,AISI_4340_52_HRC,33,-8984.507617,3588.932373,-7.025745,[-10689.99346380189]


In [40]:
pd.DataFrame(results)

,material,qtd_dados,best_y,new_candidates_vc,new_candidates_rake_angle
0,AISI_1045,14,-3822.697656,4958.335938,-5.398835
1,AISI_1045,27,-3822.697656,4125.320312,-9.665945
2,AISI_1045,40,-3822.697656,4634.667480,3.440369
3,AISI_1045,53,-3784.288184,3277.299805,7.149837
4,AISI_1045,66,-3673.538818,3662.308594,-4.311271
5,AISI_1045,79,-3673.538818,4358.854980,9.250538
6,AISI_1045,92,-3673.538818,4084.629395,-2.673907
7,AISI_1045,105,-3673.538818,4197.573242,3.582338
8,AISI_1045,118,-3673.538818,3449.481934,-5.836879
9,AISI_1045,131,-3666.605664,4075.169434,5.918156


A otimização bayesiana é para ser utilizada em um processo iterativo, onde usamos o mecanismo de geração de dados real para seguir com a busca pelos candidatos mais indicados pelo algoritmo e retro-alimentamos a otimização.
Como ainda não está um processo afinado para plugar no abaqus, vamos conectar como se o mecanismo de geração de dados real fosse o modelo lgbm_v0.

In [35]:
input_test={"depth":5.000000e-01,
"vc":3.000000e+03,
"rake_angle":-1.000000e+01,
"clearance_angle":5.000000e+00,
"expected_cut_force":6.063299e+03,
"A":5.530000e+02,
"B":6.000000e+02,
"C":1.340000e-02,
"n":2.340000e-01,
"m":1.000000e+00,
"densidade":2.700000e-09,
"young_modulus":2.100000e+05,
"poisson_coef":2.690000e-01,}

In [36]:
input_data=pd.DataFrame.from_dict([input_test])
def build_input(controlable_input, fixed_input,construct_dict):
    final_input_df=pd.DataFrame()
    for input in controlable_input:
        for counter,parameter in enumerate(construct_dict.keys()):
            fixed_input[parameter]=input[counter]
        final_input_df=pd.concat([final_input_df,fixed_input])
    return final_input_df

In [37]:
construct_dict={"rake_angle":{"sequence_len":4,"max_bound":10,"lower_bound":-10,"type":"int"},
                "vc":{"sequence_len":10,"max_bound":500,"lower_bound":30,"type":"int"},
                #"depth":{"sequence_len":4,"max_bound":2,"lower_bound":0.1,"type":"float"}
                }

In [73]:
def objective_func(x):
    #x -> [[rake_angle,vc]] 
    minimize=-1
    model=load_model("models/lgbm_v0")
    inputs=build_input(x,input_data,construct_dict)
    return minimize*predict_model(model,inputs).drop_duplicates()["prediction_label"].values

In [74]:
bounds = torch.tensor([[-15,3000.],[15,5000.]])
candidates_list=[]
cv=TimeSeriesSplit(10)
best_list=[]

for train, test in cv.split(df[["vc","rake_angle"]]):
    init_x=torch.tensor(df.loc[train][["vc","rake_angle"]].values, dtype=torch.double)
    init_y=torch.tensor(-df.loc[train]["expected_cut_force"].values,dtype=torch.double)
    init_y=init_y.unsqueeze(-1)
    best_init_y=init_y.max().item()

    n_runs=20
    for i in range(n_runs):
        print(f"Nr. of optimization run: {i}")
        new_candidates = get_next_points(init_x, init_y, best_init_y, bounds, 1)
        new_results = torch.tensor([objective_func([new_candidates[0].detach().numpy()])],dtype=torch.double)
    
        print(f"New candidates are: {new_candidates}")
        init_x = torch.cat([init_x, new_candidates])
        init_y = torch.cat([init_y,new_results])
    
        best_init_y = init_y.max().item()
        print(f"Best point performs this way: {best_init_y}")

Nr. of optimization run: 0
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[1.2480e+00, 4.6253e+03]])
Best point performs this way: -3822.69765625
Nr. of optimization run: 1
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[ -10.3055, 4988.9956]])
Best point performs this way: -3822.69765625
Nr. of optimization run: 2
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[   8.5129, 3064.8948]])
Best point performs this way: -3822.69765625
Nr. of optimization run: 3
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[  -7.7397, 4064.5984]])
Best point performs this way: -3822.69765625
Nr. of optimization run: 4
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[   6.8723, 3726.9326]])
Best point performs this way: -3822.69765625
Nr. of optimization run: 5
Transformation Pipeline and Model Successfully Loaded
New candidates are: te

c:\Users\99823827\Anaconda3\envs\Dissertacao_nightly\lib\site-packages\botorch\fit.py:171: RuntimeWarning: All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
  warn(str(err), RuntimeWarning)


Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[  -5.1825, 4975.6270]])
Best point performs this way: -3822.69765625
Nr. of optimization run: 15
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[  10.9336, 3444.7729]])
Best point performs this way: -3822.69765625
Nr. of optimization run: 16
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[   7.7912, 3089.3564]])
Best point performs this way: -3822.69765625
Nr. of optimization run: 17
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[ -13.9230, 4430.6357]])
Best point performs this way: -3822.69765625
Nr. of optimization run: 18
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[   4.6122, 3178.9141]])
Best point performs this way: -3822.69765625
Nr. of optimization run: 19
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[  11.4361, 4801.1

c:\Users\99823827\Anaconda3\envs\Dissertacao_nightly\lib\site-packages\botorch\fit.py:171: RuntimeWarning: All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
  warn(str(err), RuntimeWarning)


Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[  10.1312, 4762.3462]])
Best point performs this way: -3673.538818359376
Nr. of optimization run: 13
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[3.6325e+00, 4.1399e+03]])
Best point performs this way: -3673.538818359376
Nr. of optimization run: 14
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[-2.7224e+00,  4.1646e+03]])
Best point performs this way: -3673.538818359376
Nr. of optimization run: 15
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[  13.6487, 4584.0410]])
Best point performs this way: -3673.538818359376
Nr. of optimization run: 16


...

Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[  -8.1758, 3589.4531]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 13
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[  -5.1952, 3104.1016]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 14
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[-3.0278e+00,  4.9243e+03]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 15
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[   3.3808, 3083.6987]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 16
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[  -8.8793, 3292.9644]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 17
Transformation Pipeline and Model Successfully Loaded
New candidates are: te

c:\Users\99823827\Anaconda3\envs\Dissertacao_nightly\lib\site-packages\botorch\fit.py:171: RuntimeWarning: All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
  warn(str(err), RuntimeWarning)


Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[  -9.4509, 4908.8013]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 0
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[   8.3106, 4782.5444]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 1
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[-2.5683e+00,  3.5683e+03]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 2
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[  -4.2474, 3976.6968]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 3
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[-3.6108e+00,  4.5334e+03]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 4
Transformation Pipeline and Model Successfully Loaded
New candidates are: ten

c:\Users\99823827\Anaconda3\envs\Dissertacao_nightly\lib\site-packages\botorch\fit.py:171: RuntimeWarning: All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
  warn(str(err), RuntimeWarning)


Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[   6.7007, 3089.1987]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 9
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[  10.2898, 3995.6252]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 10
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[-2.0269e+00,  3.8549e+03]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 11
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[   7.5479, 3192.8147]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 12
Transformation Pipeline and Model Successfully Loaded
New candidates are: tensor([[-5.8334e-01,  4.8550e+03]])
Best point performs this way: -725.4552856445314
Nr. of optimization run: 13
Transformation Pipeline and Model Successfully Loaded
New candidates are: